# Finding relevant code changes in frameworks and packages

The goal of this notebook is to track evolving code bases by first extracting changes made via the git log. These can then be filtered for the correct timeframe and relevant functions. The next step would then be to analyse the changes and decide whether they are relevant to a developer that uses that part of the code for differential testing or not.

## Imports

In [22]:
import os
import inspect
import pandas as pd
from datetime import date, timedelta
import sys
import subprocess
from IPython.display import display, HTML
from tqdm import tqdm

import numpy as np
#from scipy import stats

## Setup: User Input

* The user inputs the package that they would like to update and the Deep Learning Library. 
* They then inputs the current version of the package that the DLL is using and the one that they would like to upgrade to (default: most recent version). The version is here simplified to release dates for now, since this is easier to handle for git diff.
* If the Github Link for that package is not stored, they then input the Github Link for that package.


In [69]:
# Input 1: Package name
package_name = 'tensorflow_1.12.0'

# Input 2: Deep Learning Library name and directory
dll_name = 'tensorflow_1.12.0'
dll_directory = 'A:/BachelorThesis/DLL_Testing_Tool/DL_Libraries/Tensorflow/tensorflow-1.12.0/tensorflow/python/'

# Input 3: Current version (i.e. date for simplicity) of the package (and optionally the desired version)
current_version_date = date(2021,1,1)
desired_version_date = date.today()

# Input 4: Github Link of package (if not stored by the tool)
git_url = "https://github.com/tensorflow/tensorflow.git"
#git_url = 'https://github.com/keras-team/keras.git'

In [3]:
# Input 1: Package name
package_name = 'scipy'

# Input 2: Deep Learning Library name and directory
dll_name = 'theano'
dll_directory = 'A:/BachelorThesis/DLL_Testing_Tool/DL_Libraries/Theano-rel-1.0.3/theano'

# Input 3: Current version (i.e. date for simplicity) of the package (and optionally the desired version)
current_version_date = date(2018,1,1)
desired_version_date = date.today()

# Input 4: Github Link of package (if not stored by the tool)
git_url = 'https://github.com/scipy/scipy.git'

In [5]:
# Import the package that should be upgraded (used to find the files where extracted functions are defined)
from tensorflow import keras
#import keras
import scipy

A:\Programs\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
A:\Programs\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
A:\Programs\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
A:\Programs\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or 

### Tools internal processing of the inputs

In [6]:
# TODO Check inputs for validity (i.e. does dll directory exist, is date in the correct format, is package known (for git url))

# Setup folder names
clone_folder_name = 'temp_bare_clone_' + package_name

## Create a bare clone of the library, which only includes repository data

In this way, we do not have to download the code, but still get access to the commit log.

In [7]:
# create a temporary directory for a bare clone of a give library
try:
    os.mkdir(clone_folder_name)
except:
    pass

In [8]:
# Only execute this if the clone was not yet created
if len(os.listdir(clone_folder_name)) == 0:

    # create the bare clone
    !git clone --bare {git_url} {clone_folder_name}

Cloning into bare repository 'temp_bare_clone_tensorflow_1.12.0'...


In [9]:
%cd {clone_folder_name}

A:\BachelorThesis\DLL_Testing_Tool\Code\2_Commit_Extraction_and_Analysis\temp_bare_clone_tensorflow_1.12.0


## Import the extraction data 

In [13]:
# import extracted test case data
df = pd.read_csv('../../1_Test_Case_Extraction_and_Analysis/extracted_data/{}_data.csv'.format(dll_name))

## Filter for only functions of the package




In [75]:
# For tensorflow 1.12.0, comment this line in:
package_name = 'keras'

# As a temporary solution, we will filter these for functions that contain 'package_name.' specifically
column_to_filter = 'Differential_Test_Function'
filter_keyword = package_name + '\.'

relevant_test_cases = df[df[column_to_filter].str.contains(filter_keyword, na=False)]
relevant_test_cases_unique = relevant_test_cases.Differential_Test_Function.unique()

# For demonstration: Test cases found in rnn_test.py (TF 1.12.0):
demo_test_cases = relevant_test_cases[relevant_test_cases.File_Path.str.contains(os.sep + 'rnn_test.py', regex=False)]
#demo_extracted_functions = demo_test_cases.Differential_Test_Function.unique()
#relevant_test_cases = demo_test_cases

package_name = 'tensorflow'

relevant_test_cases

,File_Path,Line_Number,Found_in_Function,Function_Definition_Line_Number,Assert_Statement_Type,Oracle_Argument_ Position,Differential_Function_Line_Number,Differential_Test_Function
13418,keras\backend_test.py,55,compare_single_input_op_to_numpy,34,assert_allclose,1,52,keras.backend.eval
13423,keras\backend_test.py,84,compare_single_input_op_to_numpy,34,assert_allclose,1,52,keras.backend.eval
13427,keras\backend_test.py,141,test_learning_phase_scope,135,assertEqual,2,137,keras.backend.learning_phase
13428,keras\backend_test.py,145,test_learning_phase_scope,135,assertEqual,2,137,keras.backend.learning_phase
13429,keras\backend_test.py,149,test_learning_phase_scope,135,assertEqual,2,137,keras.backend.learning_phase
...,...,...,...,...,...,...,...,...
26322,kernel_tests\rnn_test.py,636,testRNNCellSerialization,601,assertAllClose,1,613,keras.models.Model
26323,kernel_tests\rnn_test.py,636,testRNNCellSerialization,601,assertAllClose,1,610,keras.Input
26327,kernel_tests\rnn_test.py,636,testRNNCellSerialization,601,assertAllClose,2,613,keras.models.Model
26328,kernel_tests\rnn_test.py,636,testRNNCellSerialization,601,assertAllClose,2,633,keras.models.Model


## Getting a git diff of the current version of the extracted function and the desired version.

Procedure:
1. For a single extracted function, get the file it is defined in
2. Use git log to extract the commit id of the current version and the desired version
3. Perform a git diff, comparing the extracted file in those two commits  
4. (Selecting only the parts of the git diff that concern the extracted function)

In [77]:
def get_function_file_location(extracted_function, _package_name='tensorflow'):
    """For step 1. Find where the function is defined."""
    
    # use the extracted_function string as if it were code, since 'inspect' can't deal with strings
    str_to_execute = 'extracted_function_file_location = inspect.getsourcefile({})'.format(extracted_function)
    
    # get local scope (necessary since exec does not work well inside of function definition scopes)
    lcls = locals()
    
    # execute the string as if it were code, setting the file location variable in the local scope
    exec(str_to_execute, globals(), lcls)
    
    # getting the variable from the local scope
    extracted_function_file_location = lcls["extracted_function_file_location"]
    
    # remove the package root to get the relative file path 
    package_root_index = extracted_function_file_location.index(_package_name)
    extracted_function_file_location = extracted_function_file_location[package_root_index:]
    
    return extracted_function_file_location


def get_nearest_commit(version_date, extracted_function_file_location):
    """For step 2. Return commit ID and message of the nearest commit on or before version_date."""
    git_log_output = ''
    days = 1
    while git_log_output == '':
        git_log_command = ["git", "log", "--since", (version_date-timedelta(days=days)).strftime("%d-%m-%Y"), "--until", version_date.strftime("%d-%m-%Y"), "--", extracted_function_file_location]
        git_log_output = subprocess.run(git_log_command, stdout=subprocess.PIPE).stdout.decode('utf-8')
        
        #print("-" + str(days) + " " + git_log_output)
        
        days += 1
        
        # exit condition for when search takes too long
        if days > 100:
            return 'ERROR', 'No commit within 100 days of the entered date.'
            

    commit_id = git_log_output[7:].splitlines()[0]
    
    commit_message_command = ["git", "log", "--format=%B", "-n", "1", commit_id]
    commit_message = subprocess.run(commit_message_command, stdout=subprocess.PIPE).stdout.decode('utf-8')
    
    return commit_id, commit_message


def format_line_beginning(line):
    line_beginning = []
    for char in line:
        if char == ' ':
            line_beginning.append('&nbsp')
        else:
            break

    separator = ' '
    formatted_line = separator.join(line_beginning)
    formatted_line += line.lstrip()
    
    return formatted_line


tool_output = open("../tool_output/tool_output_{}.html".format(dll_name), "w+", encoding='utf-8')
tool_output.write("""
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    .collapsible {
      background-color: #777;
      color: white;
      cursor: pointer;
      padding: 18px;
      width: 100%;
      border: none;
      text-align: left;
      outline: none;
      font-size: 15px;
    }

    .active, .collapsible:hover {
      background-color: #555;
    }

    .content {
      padding: 0 18px;
      display: none;
      overflow: hidden;
      background-color: #f1f1f1;
    }
    </style>
    </head>
    <body>\n
""")

error_list = []
extr_func_file_location_list = []

for extracted_function in relevant_test_cases.Differential_Test_Function:

    # 1:   
    try:
        extracted_function_file_location = get_function_file_location(extracted_function, _package_name=package_name)
        #print(extracted_function_file_location)
    except Exception as exc:
        error_list.append(extracted_function + " : " + str(exc))
        extr_func_file_location_list.append("ERROR")
        continue
    
    extr_func_file_location_list.append(extracted_function_file_location)
    
relevant_test_cases.loc[:, 'Extracted_Function_File_Location'] = extr_func_file_location_list
#.insert(-1, 'Extracted_Function_File_Location', extr_func_file_location_list)

#display(relevant_test_cases)


for extracted_function_file_location in tqdm(relevant_test_cases.Extracted_Function_File_Location.unique()):
    
    tool_output.write("_____________________________________" + extracted_function_file_location + "_________________________________________\n")
    
    tool_output.write(relevant_test_cases[relevant_test_cases['Extracted_Function_File_Location'] == extracted_function_file_location].to_html())
    
    
    # 2:
    commit_id_current, commit_message_current = get_nearest_commit(current_version_date, extracted_function_file_location)
    tool_output.write("\n <br>Commit id closest to current version: " + commit_id_current + "\n")
    tool_output.write("\n <br>Commit message: " + commit_message_current.replace('\n', '<br>') + "\n")
    
    commit_id_desired, commit_message_desired = get_nearest_commit(desired_version_date, extracted_function_file_location)
    tool_output.write("<br>Commit id closest to desired version: " + commit_id_desired + "\n")
    tool_output.write("\n <br>Commit message: " + commit_message_desired.replace('\n', '<br>') + "\n<br>")
    

    # 3:
    git_diff_command = ["git", "diff", commit_id_current, commit_id_desired, "--", extracted_function_file_location]

    git_diff_output = subprocess.run(git_diff_command, stdout=subprocess.PIPE).stdout.decode('utf-8')
    
    git_diff_processed = ''
    for line in git_diff_output.splitlines():
        if line.startswith('-'):
            line = line[1:]
            git_diff_processed += "<span style=\"color:red\">- " + format_line_beginning(line) + "</span>\n"
        
        elif line.startswith('+'):
            line = line[1:]
            git_diff_processed += "<span style=\"color:green\">+" + format_line_beginning(line) + "</span>\n"
        
        elif line.startswith(' '):
            git_diff_processed += format_line_beginning(line) + "\n"
            
        else:
            git_diff_processed += line + "\n"
    
    # formatting for html
    git_diff_processed = git_diff_processed.replace('\n', '\n<br>')#.replace(' ', '&nbsp ')
    
    # add git diff as collapsible section
    tool_output.write("<button type=\"button\" class=\"collapsible\">Git Diff</button>\n<div class=\"content\">\n<p>" + git_diff_processed + "</p>\n</div>\n<br><br><br>")

# Add script to html to make git diff collapsible
tool_output.write("""
<br>
<script>
var coll = document.getElementsByClassName("collapsible");
var i;

for (i = 0; i < coll.length; i++) {
  coll[i].addEventListener("click", function() {
    this.classList.toggle("active");
    var content = this.nextElementSibling;
    if (content.style.display === "block") {
      content.style.display = "none";
    } else {
      content.style.display = "block";
    }
  });
}
</script>
</body>
</html>""")
tool_output.close()
print(str(len(error_list)) + " errors: " + str(error_list))

A:\Programs\Python36\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
100%|██████████████████████████████████████████| 25/25 [03:09<00:00,  7.59s/it]

91 errors: ["keras.backend.array_ops.placeholder : module 'tensorflow._api.v1.keras.backend' has no attribute 'array_ops'", "keras.backend.array_ops.placeholder : module 'tensorflow._api.v1.keras.backend' has no attribute 'array_ops'", "keras.backend.local_conv : module 'tensorflow._api.v1.keras.backend' has no attribute 'local_conv'", "keras.backend.local_conv : module 'tensorflow._api.v1.keras.backend' has no attribute 'local_conv'", "keras.backend.local_conv : module 'tensorflow._api.v1.keras.backend' has no attribute 'local_conv'", "keras.backend.local_conv1d : module 'tensorflow._api.v1.keras.backend' has no attribute 'local_conv1d'", "keras.backend.local_conv2d : module 'tensorflow._api.v1.keras.backend' has no attribute 'local_conv2d'", "keras.layers.local.make_2d : module 'tensorflow._api.v1.keras.layers' has no attribute 'local'", "keras.layers.deserialize : module 'tensorflow._api.v1.keras.layers' has no attribute 'deserialize'", "keras.layers.serialize : module 'tensorflow._

## Install section (for testing)

In [ ]:
!python --version

In [ ]:
# install the package (TODO if not already installed)
!{sys.executable} -m pip install {package_name}==2.2.4

In [7]:
!{sys.executable} -m pip install theano==1.0.3

  Running setup.py bdist_wheel for theano: started
  Running setup.py bdist_wheel for theano: finished with status 'done'
  Stored in directory: C:\Users\Alex\AppData\Local\pip\Cache\wheels\10\82\05\9ef5e43bfcf906b4810f85f91b09d6daf7ad213d30179defa9
Successfully built theano


You are using pip version 18.1, however version 21.1.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
%pip install tensorflow==1.12.0

In [ ]:
!{sys.executable} -m pip show keras

In [6]:
!{sys.executable} --version

Python 3.6.8


In [ ]:
sys.executable

In [3]:
!python -V

Python 3.9.4


In [31]:
inspect.getsourcefile(scipy.linalg.cholesky)

'A:\\Programs\\Python36\\lib\\site-packages\\scipy\\linalg\\decomp_cholesky.py'

## TESTING SECTION:

In [ ]:
# OLD CODE GIT LOG SECTION

# 1:

# get extracted function as string
#extracted_function = relevant_test_cases.iloc[case_id]['Differential_Test_Function']

# get the package root and remove it from the file path. This relative file path is necessary for a git diff
#package_root = ''
#exec('package_root = inspect.getsourcefile({})'.format(package_name))
# remove the init.py part from the path
#package_root = package_root.replace('__init__.py', '')
#print(package_root)


In [83]:
!start .
#os.system("git log --oneline -- {extracted_function_file_location}")
# --since "20-06-2021 00:00:00" -p

In [ ]:
# helper for finding where functions are defined
print(inspect.getsourcefile(stats.t.logpdf) + "\n")
print(inspect.getsource(stats.t.logpdf))

In [ ]:
# Different git urls:
#git_url = "https://github.com/pytorch/pytorch.git"
#git_url = "https://github.com/scipy/scipy.git"
#git_url = "https://github.com/keras-team/keras.git"

### Testing git log functions

-p shows the diffs

Hunks of differences are in the format @@ from-file-range to-file-range @@ [header].  
The from-file-range is in the form -\<start line\>,\<number of lines\>, and to-file-range is +\<start line\>,\<number of lines\>

In [ ]:
#command = ["git", "log", "--oneline", "--name-only", "--since", current_version_date, "--until", desired_version_date, "--", extracted_function_file_location]
#command = ["git", "log", "--oneline", "--since", current_version_date, "--until", desired_version_date, "--", extracted_function_file_location]
#command = ["git", "log", "--oneline", "--since", current_version_date-timedelta(days=1), "--until", current_version_date, "--", extracted_function_file_location]
#command = ["git", "log", "--oneline", "--", extracted_function_file_location]

In [64]:
!git log --oneline -- tensorflow\\python\\keras\\layers\\recurrent.py

a9cf3a0e4b4 Merge pull request #48725 from amogh7joshi:patch-3
3eee56fc4b6 Update recurrent Keras layers with less cryptic messages
1223335a8d3 Add negative parameter validation for recurrent layers.
da96383680c Fix the backward RNN layer with ragged inputs.
445bf701fbd Update keras to use tf_logging.warning instead of warn() which is deprecated.
993403945ef Remove a large amount of PY2 compatibility code.
e57dac4793d Remove the unnecessary walkaround the NoDependency in LSTM, since py2 support is ended.
7716b719e54 Slight rewording in Keras Layers RNN docs: backward compatible reason --> backward compatibility
17c5b631b96 [*.py,tensorflow/cc/framework/cc_op_gen.cc] Rename "Arguments:" to "Args:"
9a74ca84057 Improve recurrent layers Call API mask documentation
abbd5deaf18 Update Keras wrt test failure in OSS keras.
e77b0dc6ffc Add property that allows layers to specify that the input_spec can also be used as the layer call function's input_signature.
9278b9421f6 Update keras to use col

In [43]:
extracted_function_file_location

'keras\\layers\\recurrent.py'

In [41]:
!git log --since="3 hours ago" --pretty=oneline

a254474924a548867f53a0ca12aa23f81a7162bd Enable MLIR generated Mul GPU kernels for unsigned types.


In [42]:
!git log --name-only --date=local --since "20-06-2021 00:00:00" 

commit a254474924a548867f53a0ca12aa23f81a7162bd
Author: Adrian Kuegel <akuegel@google.com>
Date:   Mon Jul 26 13:22:53 2021

    Enable MLIR generated Mul GPU kernels for unsigned types.
    
    They can use the same generated code as for the signed types of the same size.
    
    PiperOrigin-RevId: 386841483
    Change-Id: Icce7317fd92586a8d1c2dad728140fc777ea9c93

tensorflow/core/kernels/cwise_op_gpu_mul.cu.cc
tensorflow/core/kernels/cwise_op_mul_1.cc
tensorflow/core/kernels/cwise_op_mul_2.cc
tensorflow/core/kernels/mlir_generated/BUILD
tensorflow/core/kernels/mlir_generated/gpu_binary_ops_test.cc
tensorflow/core/kernels/mlir_generated/gpu_op_mul.cc

commit ff6a238cfa26e0bbbcab2eabaec9a2b991078bd9
Author: Adrian Kuegel <akuegel@google.com>
Date:   Mon Jul 26 12:06:22 2021

    Enable MLIR generated Sub GPU kernels for unsigned types.
    
    They can use the same generated code as for the signed types of the same size.
    Also remove the template instantiation for uint8 and uint1

tensorflow/compiler/xla/layout_util.cc
tensorflow/compiler/xla/layout_util.h
tensorflow/compiler/xla/literal.cc
tensorflow/compiler/xla/literal.h
tensorflow/compiler/xla/literal_comparison.cc
tensorflow/compiler/xla/literal_test.cc
tensorflow/compiler/xla/literal_util.cc
tensorflow/compiler/xla/literal_util.h
tensorflow/compiler/xla/metric_table_report.cc
tensorflow/compiler/xla/metric_table_report.h
tensorflow/compiler/xla/overflow_util.h
tensorflow/compiler/xla/packed_literal_reader.cc
tensorflow/compiler/xla/permutation_util.cc
tensorflow/compiler/xla/pjrt/gpu_device.cc
tensorflow/compiler/xla/pjrt/gpu_multistream_test.cc
tensorflow/compiler/xla/pjrt/pjrt_client.h
tensorflow/compiler/xla/pjrt/pjrt_stream_executor_client.cc
tensorflow/compiler/xla/pjrt/pjrt_stream_executor_client.h
tensorflow/compiler/xla/pjrt/semaphore.cc
tensorflow/compiler/xla/pjrt/semaphore.h
tensorflow/compiler/xla/pjrt/tfrt_cpu_pjrt_client.cc
tensorflow/compiler/xla/pjrt/tfrt_cpu_pjrt_client.h
tensorflow/compil


tensorflow/core/ops/tpu_configuration_ops.cc

commit 12e1c05babba400f0f8d00d83117d3dca0a829e6
Author: A. Unique TensorFlower <gardener@tensorflow.org>
Date:   Sat Jul 24 00:17:52 2021

    Update ops-related pbtxt files.
    
    PiperOrigin-RevId: 386541933
    Change-Id: I9b44a189a56a41ba33159b4ccfce0c95748260a3

tensorflow/core/ops/compat/ops_history_v2/Batch.pbtxt
tensorflow/core/ops/compat/ops_history_v2/BatchFunction.pbtxt
tensorflow/core/ops/ops.pbtxt

commit 40a96526b0ae4f59d0d0ddf259922467f1b305a9
Author: Mingming Liu <mingmingl@google.com>
Date:   Sat Jul 24 00:16:44 2021

    Mark Batch ops a distributed runtime op
    
    PiperOrigin-RevId: 386541751
    Change-Id: I200fac4102cfd2c232a084bc591efc70a1c36cea

tensorflow/core/ops/batch_ops.cc

commit 18f339d3852a6fbeef2823426afb0f3fedeb446c
Author: Qiao Zhang <zhangqiaorjc@google.com>
Date:   Sat Jul 24 00:03:16 2021

    [PJRT:TFRT:CPU] Remove an expensive call to BufferAssignment::GetUniqueTopLevelOutputSlice in CreateBuff

    
    PiperOrigin-RevId: 386260449
    Change-Id: Idd941cc79a17724ab2a6a7f208d3f398674a1e49

tensorflow/compiler/mlir/tfrt/transforms/tpu_passes.h

commit 80d2dda7129ad5810d29ccc171d893aa9a0379b2
Merge: ca2c4502975 28db1e8e3b2
Author: TensorFlower Gardener <gardener@tensorflow.org>
Date:   Thu Jul 22 18:05:37 2021

    Merge pull request #50876 from ROCmSoftwarePlatform:proper_fix_for_xla_unit_test_seg_fault_on_hip
    
    PiperOrigin-RevId: 386251135
    Change-Id: I7a9463f07d1c34e8d199e8729e51a127d3172605

commit ca2c45029753b46a55240f7575396b16f6aecea8
Author: Blake Hechtman <blakehechtman@google.com>
Date:   Thu Jul 22 17:03:42 2021

    [XLA] Don't run a pass if it has not been changed in more than 3 fixed pass iterations.
    
    PiperOrigin-RevId: 386241029
    Change-Id: I5de87bb2f773a2e29b445eae7ddd5c8b52f06d4a

tensorflow/compiler/xla/service/BUILD
tensorflow/compiler/xla/service/hlo_pass_fix.h
tensorflow/compiler/xla/service/hlo_pass_interface.h
tensorflow/compiler/xla/

tensorflow/core/platform/cloud/gcs_dns_cache.cc
tensorflow/core/platform/cloud/gcs_dns_cache.h
tensorflow/core/platform/cloud/gcs_dns_cache_test.cc
tensorflow/core/platform/cloud/gcs_file_system.cc
tensorflow/core/platform/cloud/gcs_file_system_test.cc
tensorflow/core/platform/cloud/gcs_throttle.cc
tensorflow/core/platform/cloud/gcs_throttle_test.cc
tensorflow/core/platform/cloud/http_request.h
tensorflow/core/platform/cloud/oauth_client.cc
tensorflow/core/platform/cloud/ram_file_block_cache.cc
tensorflow/core/platform/cloud/ram_file_block_cache.h
tensorflow/core/platform/cloud/time_util_test.cc

commit fb12f3df6f321d9f223ccf6d129524e9821a048a
Author: Fergus Henderson <fergus@google.com>
Date:   Tue Jul 20 17:57:45 2021

    Promote C++ InterpreterBuilder::AddDelegate method from experimental to stable.
    
    Rationale:
    
    1. Equivalent functionality is already non-experimental
    in the TF Lite C API via the TfLiteIntepreterOptionsAddDelegate
    function.
    
    2. This i

...

    
    PiperOrigin-RevId: 382419639
    Change-Id: I149d67b84ee5fef30b5a1426081a40b64fd5ca8e

tensorflow/lite/java/src/main/java/org/tensorflow/lite/Interpreter.java
tensorflow/lite/java/src/main/java/org/tensorflow/lite/InterpreterApi.java

commit 3fdb2ef39177418fdae32689e9d9a995cbc56c46
Author: A. Unique TensorFlower <gardener@tensorflow.org>
Date:   Thu Jul 1 02:23:16 2021

    Update TFRT dependency to use revision
    http://github.com/tensorflow/runtime/commit/c32c57da845860c3cd74b290757358eb0791b329.
    
    PiperOrigin-RevId: 382418766
    Change-Id: I1997070781b014e4b4d8955a1448586835e4e39f

third_party/tf_runtime/workspace.bzl

commit ebf76d2c1e3a24e84565d5b3ea7fc7f944924a97
Author: Skye Wanderman-Milne <skyewm@google.com>
Date:   Thu Jul 1 02:19:44 2021

    [XLA:PJRT][XLA:Python] Add TPU executable de/serialization methods.
    
    The methods are added to PjRtClient and not PjRtExecutable since only
    the client has subclasses for each platform, and we need to use
  

tensorflow/compiler/mlir/lite/transforms/prepare_quantize.cc
tensorflow/compiler/mlir/lite/transforms/prepare_tf.cc
tensorflow/compiler/mlir/lite/transforms/quantize.cc
tensorflow/compiler/mlir/lite/transforms/raise_custom_ops.cc
tensorflow/compiler/mlir/lite/transforms/runtime_verify.cc
tensorflow/compiler/mlir/lite/transforms/split_merged_operands.cc
tensorflow/compiler/mlir/lite/transforms/trim_functions_tf.cc
tensorflow/compiler/mlir/lite/transforms/while_loop_outline.cc
tensorflow/compiler/mlir/tensorflow/transforms/cluster_formation.cc
tensorflow/compiler/mlir/tensorflow/transforms/cross_host_transfer.cc
tensorflow/compiler/mlir/tensorflow/transforms/executor_island_coarsening.cc
tensorflow/compiler/mlir/tensorflow/transforms/guarantee_all_funcs_one_use.cc
tensorflow/compiler/mlir/tensorflow/transforms/parallel_execute_to_islands.cc
tensorflow/compiler/mlir/tensorflow/transforms/test_cluster_ops_by_policy.cc
tensorflow/compiler/mlir/tensorflow/transforms/tf_data_optimization_pass

tensorflow/lite/delegates/flex/BUILD
tensorflow/lite/delegates/flex/build_def.bzl
tensorflow/lite/delegates/flex/delegate.cc
tensorflow/lite/delegates/flex/delegate_symbol.cc

commit 068c75c6d4e570cca38f8d650ca6d34c9f49d878
Author: Chenkai Kuang <chenkai@google.com>
Date:   Fri Jun 25 02:04:44 2021

    Renable SyncOnRead variable test in mirrored_variable_test.
    
    PiperOrigin-RevId: 381367893
    Change-Id: If4a0c546c92c040cec5495c1686def028f62fc62

tensorflow/python/distribute/mirrored_variable_test.py

commit f9d39e29c117cfb564711b36345a3392846ce0ce
Author: Andrew Selle <aselle@google.com>
Date:   Fri Jun 25 02:02:47 2021

    Make initial MLIR transform API features (loading saved)
    
    PiperOrigin-RevId: 381367494
    Change-Id: I7a491fe0d65f284e8de3d7b7331d084c32640ecf

tensorflow/compiler/mlir/tensorflow/BUILD

commit d94936be4029678850a4708c81dcb41c2bb501c8
Author: Andrew Audibert <aaudibert@google.com>
Date:   Fri Jun 25 01:48:01 2021

    [tf.data] Add support for s


    [tf.data service] Overhaul module-level tf.data service docs.
    
    PiperOrigin-RevId: 381056918
    Change-Id: I9d07b191a276116172d27ab670694183e38002e0

tensorflow/python/data/experimental/service/__init__.py

commit 2c79273b135655d24dc200820a9f86fd71d1695a
Author: Yang Chen <yangchen@google.com>
Date:   Wed Jun 23 18:53:45 2021

    [tf.data service] Fix windows unit tests.
    
    Windows tests can't handle grpc+local protocol.
    
    PiperOrigin-RevId: 381054102
    Change-Id: Ic48e155b68421686cb9e4039edbac1b9646b2b9f

tensorflow/core/data/service/BUILD
tensorflow/core/data/service/data_service_test.cc
tensorflow/core/data/service/test_cluster.cc
tensorflow/core/data/service/worker_client_test.cc

commit 3ec055ca5d665a698d800ee2f964b8d7db3a6cca
Author: Stella Laurenzo <laurenzo@google.com>
Date:   Wed Jun 23 18:53:45 2021

    Switch lowering of reductions with keep_dims=true to use ExpandDimsOp instead of DynamicReshapeOp.
    
    * We desire that cases of trivial uni

third_party/tf_runtime/workspace.bzl

commit 54ed7d2f7c0a3abc5f27bf41c05fe4a4167c23bd
Author: Chao Mei <chaomei@google.com>
Date:   Tue Jun 22 11:04:55 2021

    Fix a typo.
    
    PiperOrigin-RevId: 380760402
    Change-Id: I201944a5f0e3cba2deb011d6bd4085522f8a1923

tensorflow/lite/experimental/acceleration/mini_benchmark/metrics/BUILD

commit c52692f0b8b5dfa9a895dc1fb0972f9e66973a50
Author: A. Unique TensorFlower <gardener@tensorflow.org>
Date:   Tue Jun 22 11:02:26 2021

    Update GraphDef version to 805.
    
    PiperOrigin-RevId: 380760044
    Change-Id: Ia2ee099779e97c111c6f4bd10975fcc8b5caf1ca

tensorflow/core/public/version.h

commit 4889d6a7c2c7f39679d25396dfbd1b3deb653051
Author: A. Unique TensorFlower <gardener@tensorflow.org>
Date:   Tue Jun 22 11:02:25 2021

    compat: Update forward compatibility horizon to 2021-06-22
    
    PiperOrigin-RevId: 380760036
    Change-Id: I6d84583476c90db9aae69ecfc8e3803e1112272c

tensorflow/python/compat/compat.py

commit 6b900c82945d

In [ ]:
!git log --name-only --oneline --since "20-06-2021 00:00:00"

In [ ]:
!git log --name-only --oneline --since "20-06-2021 00:00:00"
#--since "20-06-2021 00:00:00" -p -- scipy/special/_basic.py